# Inicio

In [76]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

# Functions

In [74]:
headers = {
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8',
    'accept-language': 'pt-BR,pt;q=0.5',
    'cache-control': 'max-age=0',
    'priority': 'u=0, i',
    'referer': 'https://www.residentevildatabase.com/personagens/',
    'sec-ch-ua': '"Brave";v="129", "Not=A?Brand";v="8", "Chromium";v="129"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'sec-gpc': '1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/129.0.0.0 Safari/537.36',
    }

##
def get_content(url):
    
    resp = requests.get(url, headers = headers)
    return resp

##
def get_basic_infos(soup):

    div_page = soup.find('div', class_='td-page-content')
    paragrafo = div_page.find_all('p')[1]
    ems = paragrafo.find_all('em')

    data = {}
    for i in ems:
        chave, valor, *_ = i.text.split(':')
        chave = chave.strip(" ")
        data[chave] = valor.strip(" ")

    return data

##
def get_aparicoes(soup):
    lis = (soup.find('div', class_='td-page-content')
               .find('h4')
               .find_next()
               .find_all('li'))

    aparicoes = [i.text for i in lis]
    return aparicoes

##
def get_personagens_info(url):
    
    response = get_content(url)

    if response.status_code !=200:
        print('não foi possível obter os dados')
        return {}
    else:
        soup = BeautifulSoup(response.text)
        data = get_basic_infos(soup)
        data['Aparicoes'] = get_aparicoes(soup)
        return data

##
def get_links():
    url = 'https://www.residentevildatabase.com/personagens/'
    resp = requests.get(url, headers=headers)
    soup_personagens = BeautifulSoup(resp.text)
    ancoras = (soup_personagens.find('div', class_='td-page-content')
                               .find_all('a'))
    links = [i['href'] for i in ancoras]
    return links


In [85]:
links = get_links()

data =[]
for i in tqdm(links):
    d = get_personagens_info(i)
    d['Link'] = i
    nome = i.strip('/').split('/')[-1].replace('-', ' ').title()
    d['Nome'] = nome
    data.append(d)

 31%|███       | 51/164 [00:11<00:21,  5.19it/s]

não foi possível obter os dados


100%|██████████| 164/164 [01:01<00:00,  2.68it/s]


In [86]:
df = pd.DataFrame(data)
df

,Ano de nascimento,Tipo sanguíneo,Altura,Peso,Aparicoes,Link,Nome,de nascimento
0,1974 (24 anos em 1998),AB,Desconhecida.,Desconhecido.,"[Biohazard / Resident Evil 2 (1998), Biohazard...",https://www.residentevildatabase.com/ada-wong/,Ada Wong,NaN
1,Desconhecido.,Desconhecido.,Desconhecida.,Desconhecido.,[Biohazard / Resident Evil 6 (2012)],https://www.residentevildatabase.com/adam-benf...,Adam Benford,NaN
2,Desconhecido.,Desconhecido.,Desconhecida.,Desconhecido.,"[Biohazard / Resident Evil 7 (2017), Biohazard...",https://www.residentevildatabase.com/alan-doug...,Alan Douglas,NaN
3,1960,O,"1,83cm (em jogos antigos); 1,90cm (em RE5)","84,5kg (em jogos antigos); 90kg (em RE5)","[Biohazard / Resident Evil Clássico (1996), Bi...",https://www.residentevildatabase.com/albert-we...,Albert Wesker,NaN
4,Desconhecido.,Desconhecido.,"2,90m (aprox.)",Desconhecido.,[Biohazard / Resident Evil Village (2021)],https://www.residentevildatabase.com/alcina-di...,Alcina Dimitrescu,NaN
...,...,...,...,...,...,...,...,...
159,1962 (não confirmado).,O,"1,78cm","66,7kg","[Biohazard / Resident Evil 0 (2003), Biohazard...",https://www.residentevildatabase.com/william-b...,William Birkin,NaN
160,desconhecido.,desconhecido.,desconhecida.,desconhecido.,[Biohazard / Resident Evil: Operation Raccoon ...,https://www.residentevildatabase.com/willow-ca...,Willow Caroline Floyd,NaN
161,Desconhecido.,Desconhecido.,Desconhecido.,Desconhecido.,[BIOHAZARD / RESIDENT EVIL: Infinite Darkness ...,https://www.residentevildatabase.com/wilson-se...,Wilson Secretario Seguranca,NaN
162,1978 (20 anos em 1998),Desconhecido.,"1,63m",58kg,"[Biohazard / Resident Evil Outbreak (2003), Bi...",https://www.residentevildatabase.com/yoko-suzuki/,Yoko Suzuki,NaN


In [87]:
df.to_csv("dados_re.csv", index=False, sep=';')
df.to_parquet('dados_re.parquet', index=False)
df.to_pickle('dados_re.pkl')